In [4]:
import torch

t = torch.tensor([0, 1, 0, 2, 1, 2, 3, 3])
m1 = torch.tensor([True, True, False, True, False, False, True, False])
m2 = ~ m1
t[m2]

tensor([0, 1, 2, 3])

In [15]:
m1.nonzero(as_tuple=False).view(-1)[[2,1]]

tensor([3, 1])

---

In [201]:
DATA_DIR = '/home/gangda/workspace/graph_engine/temp'
PREP_DIR = '/home/gangda/workspace/graph_engine/data/ogbn-products-p4'

ppr = torch.load(DATA_DIR + '/data.pt')
ppr[0]

(tensor([613393, 581754,  29689,  ..., 159676, 399976, 440506],
        dtype=torch.int32),
 tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int8),
 tensor([6.8911e-06, 1.7770e-03, 5.6566e-05,  ..., 5.8274e-05, 1.4357e-05,
         2.1618e-05]))

In [202]:
from ogb.nodeproppred import DglNodePropPredDataset

og, y = DglNodePropPredDataset(name='ogbn-products', root='/data/gangda/dgl')[0]
X = og.ndata['feat']
edge_index = torch.load(PREP_DIR + '/dgl_edge_index.pt')
X, edge_index

(tensor([[ 0.0319, -0.1959,  0.0520,  ...,  0.0767, -0.3930, -0.0648],
         [-0.0241,  0.6303,  1.0606,  ..., -1.6875,  3.5867,  0.8182],
         [ 0.3327, -0.5586, -0.2886,  ..., -0.3716,  0.2521,  0.0415],
         ...,
         [ 0.1066,  0.2655, -0.0057,  ...,  1.0867,  0.0759, -1.1737],
         [ 0.2497, -0.2574,  0.4123,  ...,  1.5466,  1.0310, -0.2966],
         [ 0.7175, -0.2393,  0.0443,  ..., -1.0132, -0.4141, -0.0823]]),
 tensor([[      0,       0,       0,  ..., 2449028, 2449028, 2449028],
         [    384,    2412,    7554,  ..., 1787657, 1864057, 2430488]]))

In [203]:
v, i = torch.sort(ppr[0][2], descending=True)
top_100_index = i[:100]
local_id, shard_id = ppr[0][0][top_100_index], ppr[0][1][top_100_index]
local_id, shard_id

(tensor([     0, 278804, 455672, 497655, 452891, 444152, 288305,  93215, 195938,
         125016, 486015, 530822, 578054, 481287, 473931, 486758, 420232, 587793,
         505595, 474696, 292000, 450011, 259331, 572588, 271538, 437809, 432737,
         164038, 398473, 159000, 425421, 453506, 165863, 574107, 570637,  89482,
         518980, 541320, 476820, 514113, 518844, 215361,  15741, 594026, 565901,
         139082, 466278, 409354, 257575, 147013, 587127, 466706, 431252,  49016,
         609684, 133808, 194663, 579937, 348718, 434766, 552249, 622466,  81609,
         474646, 474723, 506845, 479771, 254098, 573020, 122724, 538145, 485431,
         535468, 512714, 547847, 611277, 578137, 277645, 471651, 240417,  76635,
         616164, 581754, 593143, 576630,  32276, 585355, 537633, 374098, 434405,
         394040, 409727, 349741, 390371, 339188, 136042, 432869, 517115, 446749,
         464813], dtype=torch.int32),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [198]:
parts = torch.load(PREP_DIR + '/metis_partitions.pt')
num_parts = len(parts)

part_core_global_ids = []
for i in range(num_parts):
    part = parts[i]
    core_mask = part.ndata['inner_node'].type(torch.bool)
    part_global_id = part.ndata['orig_id']
    part_core_global_ids.append(part_global_id[core_mask])

part_core_global_ids

[tensor([1578341,  580537, 1578356,  ..., 1969840, 2255870,  320072]),
 tensor([1231032, 2291684, 1306048,  ...,   10705,   51625,  930216]),
 tensor([ 920623, 1636464,  533610,  ..., 1957407, 1981728,  691187]),
 tensor([ 259846, 2060628, 2032898,  ...,  511433, 1681439, 1681440])]

In [204]:
local_id_mapping = torch.load(PREP_DIR + '/local_id_mapping.pt')
shard_id_mapping = torch.load(PREP_DIR + '/shard_id_mapping.pt')

def get_global_id(local_ids_, shard_ids_):
    local_ids_ = local_ids_.to(torch.long)
    global_ids = torch.empty_like(local_ids_)
    for j in range(num_parts):
        mask = shard_ids_ == j
        if mask.sum() == 0: continue
        global_ids[mask] = part_core_global_ids[j][local_ids_[mask]]
    return global_ids

get_global_id(local_id, shard_id)

tensor([1578341, 1040833,  196128,  232835,  218061, 2028409, 1358269, 1499731,
         880916, 1853590,   82307, 1888491,   39785,   80922,  215340,   83086,
         174582,   47004,  111744, 2301674,  733033,   95345, 1004785,  166341,
        1079953,   62190,  120665,  519920, 2337523, 1652673,  171130,  108086,
        1673204,  167416, 2385923, 1508538,  133438,  146506,  101068,   22019,
           5807, 1186379, 1541565,  141295, 1914750, 1848082,  106002,  261337,
        1001437, 1681058,   46790,  106128, 2032080, 1465257,  328131,  410859,
        1120712, 1882334, 1215430,  119741,   26137,   35549, 1527459,  107237,
          98555,  191855,  189967,  976753,  166250,  397914,   26652,  279386,
        1942497,  155362,   13538,  161245,   39748, 1038434,  107201,  951020,
        1519425,  128681,   41001,  305715, 2198315,  532943,   47469,   26696,
        1261833, 2030709,  180796,   70648,  829119,  119226, 1236241,  414299,
         177042,  145002,  198189,  2025

In [ ]:
from torch_geometric.data import Data
from torch_geometric.utils import is_undirected, contains_self_loops, add_remaining_self_loops

data = Data(X, add_remaining_self_loops(edge_index)[0])
X

In [249]:
from functools import partial
from torch_geometric.utils import subgraph

K = 100
batch_index = torch.arange(32)
rank = 0
dist_subgraph = lambda x: subgraph(x, edge_index)[0]

def convert_batch_data(ppr_res, batch_index, K, dist_subgraph):
    batch_size = batch_index.shape[0]

    batch_local_ids = [batch_index]
    batch_shard_ids = [torch.full((batch_size,), rank)]
    for i in range(len(ppr_res)):
        val, idx = torch.sort(ppr_res[i][2], descending=True)
        top_k_index = idx[:K]
        batch_local_ids.append(ppr_res[i][0][top_k_index])
        batch_shard_ids.append(ppr_res[i][1][top_k_index])

    batch_local_ids, batch_shard_ids = torch.cat(batch_local_ids), torch.cat(batch_shard_ids)
    subset = get_global_id(batch_local_ids, batch_shard_ids)
    batch_node_global_id = subset[:batch_size]

    unique_subset, inv = subset.unique(return_inverse=True)
    sub_edge_index = dist_subgraph(unique_subset)
    local_e = sub_edge_index.unique(return_inverse=True)[1]

    return Data(X[unique_subset], local_e, y=y[batch_node_global_id], ego_idx=inv[:batch_size])


batch = convert_batch_data(ppr, batch_index, K, dist_subgraph)
batch

Data(x=[3161, 100], edge_index=[2, 63482], y=[32, 1], ego_idx=[32])

1. Distributed Batch Loader: Source Node ID -> Local ID
2. Convert: local ID + shard ID to global ID
3. Construct Subgraph
4. Forward Conv
5. Compute Loss term of batch nodes
6. Backward + Sync